<a href="https://colab.research.google.com/github/BenUCL/Reef-acoustics-and-AI/blob/main/Code/CNN_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Train the CNN**

This script provides an example of training the CNN on the minibatch files which can be created with the 'CNN minibatch creation' script. This uses a small subset of the Indonesian dataset.

This outputs a csv file of predictions for each 0.96sec chunk from each audio files. These can be converted into predictions for whole minutes and a final test accuracy reported using the 'Trained CNN accuracy calculator' script - note csv's from the full datasets are provided from here on as intensive analysis with audio files is no longer needed.

# **Using Colabs free GPU feature**

Google colab provides free GPU access (with some limits), see here: https://research.google.com/colaboratory/faq.html

This can be used to significantly increase training speed. To switch this on go to 'Runtime' at the top and change type to 'GPU'.

In [3]:
# Connect your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install numpy==1.21.5 resampy==0.2.2 tensorflow==1.15 tf_slim==1.1.0 six==1.15.0 soundfile==0.10.3.post1

""" As package versions began updating this threw errors on the smoke test. 
For a faster download versions could be removed but this may throw errors. 
As of 17/10/22 it gives the below output, but, the smoketest codeblock passes:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.16.0 requires gast>=0.3.2, but you have gast 0.2.2 which is incompatible.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.15.0 which is incompatible.
Successfully installed gast-0.2.2 keras-applications-1.0.8 llvmlite-0.32.1 numba-0.49.1 numpy-1.21.5 resampy-0.2.2 soundfile-0.10.3.post1 tensorboard-1.15.0 tensorflow-1.15.0 tensorflow-estimator-1.15.1 tf-slim-1.1.0
WARNING: The following packages were previously imported in this runtime:
  [numpy]
You must restart the runtime in order to use newly installed versions. """

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 38.8 MB/s 
     |████████████████████████████████| 323 kB 66.7 MB/s 
     |████████████████████████████████| 412.3 MB 24 kB/s 
     |████████████████████████████████| 352 kB 66.3 MB/s 
     |████████████████████████████████| 503 kB 61.4 MB/s 
     |████████████████████████████████| 50 kB 7.4 MB/s 
     |████████████████████████████████| 3.8 MB 52.2 MB/s 
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320732 sha256=232a8d99227cc565dc002a942786d3bd0132535aff8aa631c90757080adcbd95
  Stored in directory: /root/.cache/pip/wheels/a0/18/0a/8ad18a597d8333a142c9789338a96a6208f1198d290ece356c
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=58b853bb969455dbf960df7097b3f2956c2030875bf5deec49ec63f19bcf0074
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c51

" As package versions began updating this threw errors on the smoke test. \nFor a faster download versions could be removed but this may throw errors. \nAs of 17/10/22 it gives the below output, but, the smoketest codeblock passes:\n\nERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.\ntensorflow-probability 0.16.0 requires gast>=0.3.2, but you have gast 0.2.2 which is incompatible.\nkapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.15.0 which is incompatible.\nSuccessfully installed gast-0.2.2 keras-applications-1.0.8 llvmlite-0.32.1 numba-0.49.1 numpy-1.21.5 resampy-0.2.2 soundfile-0.10.3.post1 tensorboard-1.15.0 tensorflow-1.15.0 tensorflow-estimator-1.15.1 tf-slim-1.1.0\nWARNING: The following packages were previously imported in this runtime:\n  [numpy]\nYou must restart the runtime in order to use newly installed versions. "

In [5]:
# Should output 'Looks good to me at the bottom!'
%cd /content/drive/MyDrive/Reef soundscapes with AI/Audioset
!python vggish_smoke_test.py

/content/drive/MyDrive/Reef soundscapes with AI/Audioset
Instructions for updating:
non-resource variables are not supported in the long term

Testing your install of VGGish

Log Mel Spectrogram example:  [[-4.47297436 -4.29457354 -4.14940631 ... -3.9747003  -3.94774997
  -3.78687669]
 [-4.48589533 -4.28825497 -4.139964   ... -3.98368686 -3.94976505
  -3.7951698 ]
 [-4.46158065 -4.29329706 -4.14905953 ... -3.96442484 -3.94895483
  -3.78619839]
 ...
 [-4.46152626 -4.29365061 -4.14848608 ... -3.96638113 -3.95057575
  -3.78538167]
 [-4.46152595 -4.2936572  -4.14848104 ... -3.96640507 -3.95059567
  -3.78537143]
 [-4.46152565 -4.29366386 -4.14847603 ... -3.96642906 -3.95061564
  -3.78536116]]
2022-10-18 11:30:20.255226: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-10-18 11:30:20.327965: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there mu

In [6]:
#From original vggish_train_demo.py script on github
from __future__ import print_function

from random import shuffle

import numpy as np
import tensorflow.compat.v1 as tf
import tf_slim as slim

import vggish_input
import vggish_params
import vggish_slim

#Modules added by Ben
import os #for handling directories
import glob #for dealing with files in dir
import pandas as pd #for saving output at end in dataframe
import sklearn
import math
import pickle
from sklearn.model_selection import train_test_split #added for train/test split
from numpy import loadtxt #addded so predictions can be output to CSV file
from datetime import datetime #added to append time to csv output file name to prevent overwriting

Instructions for updating:
non-resource variables are not supported in the long term


**Set paths to access modules and pickle files, also set CNN parameters.**

Two classes are used here, increase _NUM_CLASSES if needed. A batch size of 16 was used as larger batches can cause a memory error on colab depending on which GPU you are  allocated. The network trains for 5 epochs currently to save computation time, the final study used UCL's computing cluster to train for 50 epochs on the full datasets.

In [11]:
#which repeat of the cross-val is this? (1-8):
repeat = 1 # Used to set seed for train/val/test split

### Change paths if you re-structure folders

# Path to the location where your audio file are stored:
audio_dir = r'/content/drive/MyDrive/Reef soundscapes with AI/audio_dir' 

# Path to folder containing vggish setup files and 'AudiosetAnalysis' downloaded from sarebs supplementary
vggish_files = r'/content/drive/MyDrive/Reef soundscapes with AI/Audioset' 

# Output folder for results:
results_dir = r'/content/drive/MyDrive/Reef soundscapes with AI/Results/Colab_CNN_predictions/' 

#Set the directories where logmel-spectrograms will be stored for train, test and validation sets:
pickle_trainfiles_dir = r'/content/drive/MyDrive/Reef soundscapes with AI/Results/minibatches_train/'
pickle_valfiles_dir = r'/content/drive/MyDrive/Reef soundscapes with AI/Results/minibatches_val/'
pickle_testfiles_dir = r'/content/drive/MyDrive/Reef soundscapes with AI/Results/minibatches_test/'

#how many classes?:
_NUM_CLASSES = 2

#name a column for each class e.g 'class1', 'class2', or 'healthy', 'degraded'
col_names = 'Healthy','Degraded', 'True class'

#Batch size:
batch_size = 16 # larger batches can cause a memory error on the NN script on colab depending on which GPU you are  allocated 

# Number of epochs.
num_epochs = 5



In [8]:
#### Some final set up
# Find number of minibatches for networks for loop
minibatches = [filename for filename in os.listdir(pickle_trainfiles_dir) if filename.startswith("train_minibatch")]
num_minibatches = len(minibatches) #this takes the last digit of the last pickle files, denoting how many minibatches there are

# Get number of train/test/val minibatches
num_train_batches = int(len(os.listdir(pickle_trainfiles_dir)))
print('Number of train minibatches found: ' + str(num_train_batches))
num_val_batches = int(len(os.listdir(pickle_valfiles_dir)))
print('Number of validation minibatches found: ' + str(num_val_batches))
num_test_batches = int(len(os.listdir(pickle_testfiles_dir)))
print('Number of test minibatches found: ' + str(num_test_batches))

os.chdir(vggish_files) 

# Used to find averages of accuracy score across minibatches later
def Average(lst):
    return sum(lst) / len(lst)

print('Cross validation combination: ' + str(repeat))

Number of train minibatches found: 1
Number of validation minibatches found: 1
Number of test minibatches found: 1
Cross validation combination: 1


# **Run the neural network**

In [9]:
#RUN THIS BLOCK ONLY ONCE PER SESSION - otherwise it will error

flags = tf.app.flags

flags.DEFINE_boolean(
    'train_vggish', True,
    'If True, allow VGGish parameters to change during training, thus '
    'fine-tuning VGGish. If False, VGGish parameters are fixed, thus using '
    'VGGish as a fixed feature extractor.')

flags.DEFINE_string(
    'checkpoint', 'vggish_model.ckpt',
    'Path to the VGGish checkpoint file.')

FLAGS = flags.FLAGS

'An exception has occurred, use %tb to see the full traceback.' error will occur, fear not, this just means its finished 

In [10]:
%%timeit
"""To train 5 epochs on the 123x1min files in the of training data this process
takes up to 80 minutes on a CPU. Depending which GPU Colab provides you this can 
take < 5min on colabs GPU. The final study used NVIDIA A100 GPU's which 
generally provide the highest speed as of 2022."""

### Run the network and save the predictions and accuracy at each epoch

#

### Train NN, output results
r"""This uses the VGGish model definition within a larger model which adds two 
layers on top, and then trains this larger model. 

We input log-mel spectrograms (X_train) calculated above with associated labels 
(y_train), and feed the batches into the model. Once the model is trained, it 
is then executed on the validation and log-mel spectrograms (X_validation, 
X_test), and the accuracy is output for each.

Alongside .csv file with the predictions for each 0.96s chunk and their true
class is also output for the test data. Column1 = the logit for the first class,
Column2 = the logit for the scond class etc. The final column is the true class.

Final accuracy is actually taken from these predictions in another script which 
takes the  most common predicted class across an entire minute using each 0.96s 
chunks prediction."""




def main(X):   
  with tf.Graph().as_default(), tf.Session() as sess:
    # Define VGGish.
    embeddings = vggish_slim.define_vggish_slim(training=FLAGS.train_vggish)
    
    
    # Define a shallow classification model and associated training ops on top
    # of VGGish.
    with tf.variable_scope('mymodel'):
      # Add a fully connected layer with 100 units. Add an activation function
      # to the embeddings since they are pre-activation.
      num_units = 100
      fc = slim.fully_connected(tf.nn.relu(embeddings), num_units)

      # Add a classifier layer at the end, consisting of parallel logistic
      # classifiers, one per class. This allows for multi-class tasks.
    #  logits = slim.fully_connected(                                      ### logits threw me, would be easier to name this 'end model' or something
     #     fc, _NUM_CLASSES, activation_fn=None, scope='logits')
     # tf.sigmoid(logits, name='prediction')
    
      linear_out= slim.fully_connected(                                      
          fc, _NUM_CLASSES, activation_fn=None, scope='linear_out')
      logits = tf.sigmoid(linear_out, name='logits')
    
      # Add training ops.
      with tf.variable_scope('train'):
        global_step = tf.train.create_global_step()

        # Labels are assumed to be fed as a batch multi-hot vectors, with
        # a 1 in the position of each positive class label, and 0 elsewhere.
        labels_input = tf.placeholder(
            tf.float32, shape=(None, _NUM_CLASSES), name='labels')

        # Cross-entropy label loss.
        xent = tf.nn.sigmoid_cross_entropy_with_logits(
            logits=logits, labels=labels_input, name='xent')    ###=labels is selecting my 'y', logits is like a precursor to predictions?
        loss = tf.reduce_mean(xent, name='loss_op')
        tf.summary.scalar('loss', loss)

        # We use the same optimizer and hyperparameters as used to train VGGish.
        optimizer = tf.train.AdamOptimizer(
            learning_rate=vggish_params.LEARNING_RATE,
            epsilon=vggish_params.ADAM_EPSILON)
        train_op = optimizer.minimize(loss, global_step=global_step)

    # Initialize all variables in the model, and then load the pre-trained
    # VGGish checkpoint.
    sess.run(tf.global_variables_initializer())         ### this starts the session appaz
    vggish_slim.load_vggish_slim_checkpoint(sess, FLAGS.checkpoint)

    
    features_input = sess.graph.get_tensor_by_name(
        vggish_params.INPUT_TENSOR_NAME)
    
    # The training loop.
    #test_predictions = pd.DataFrame(columns = col_names) #this was in wrong place before
    highest_acc_score = 0
    for epoch in range(num_epochs):
            validation_accuracy_scores = []
            test_accuracy_scores = []
            test_batch_scores = []
            val_batch_scores = []
            epoch_loss = 0
            i=0
            while i < num_minibatches: 
                print('mini batch'+str(i))
                train_pickle_file = pickle_trainfiles_dir + 'train_minibatch_' + str(i)
                with open(train_pickle_file, "rb") as fp:   # Unpickling
                  batch = pickle.load(fp)
                batch_x, batch_y = zip(*batch)

                _, c = sess.run([train_op, loss], feed_dict={features_input: batch_x, labels_input: batch_y})
                epoch_loss += c
                i+=1
            #print no. of epochs and loss
            print('Epoch', epoch+1, 'completed out of', num_epochs,', loss:',epoch_loss) 

            #If these lines are left here, it will evaluate on the val and test data every iteration and print accuracy
            #note this adds a small computational cost
            correct = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_input, 1)) #This line returns the max value of each array, which we want to be the same (think the prediction/logits is value given to each class with the highest value being the best match)
            accuracy = tf.reduce_mean(tf.cast(correct, 'float')) #changes correct to type: float
            
            ## Inferencing on validation data        
            for z in range(num_val_batches):
              val_pickle_file = pickle_valfiles_dir + 'val_minibatch_' + str(z)
              with open(val_pickle_file, "rb") as fp:   # Unpickling
                 val_batch = pickle.load(fp)
              X_validation, y_validation = zip(*val_batch) # unzip the pickle output
              validation_accuracy = accuracy.eval({features_input:X_validation, labels_input:y_validation}) #inference
              val_batch_scores.append(validation_accuracy) #save accuracy score

            val_avg_score = Average(val_batch_scores) # gets the average across all val minibatches for this epoch
            validation_accuracy_scores.append(val_avg_score) # saves this average across val minibatches for the epoch
            print('Validation accuracy:', val_avg_score)#accuracy.eval({features_input:X_test, labels_input:y_test})) #TF is smart so just knows to feed it through the model without us seeming to tell it to. .eval() uses the current session which I guess is my model?
            
            if val_avg_score > highest_acc_score:
              # If the validation accuracy improved, inferencing will be done on the test data
              highest_acc_score = val_avg_score
              best_epoch = str(epoch+1)
              test_predictions = pd.DataFrame(columns = col_names)

              ## Inferencing on test data        
              for v in range(num_test_batches):
                test_pickle_file = pickle_testfiles_dir + 'test_minibatch_' + str(v)
                with open(test_pickle_file, "rb") as fp:   # Unpickling
                  test_batch = pickle.load(fp)
                X_test, y_test = zip(*test_batch) # unzip the pickle output                  
                test_accuracy = accuracy.eval({features_input:X_test, labels_input:y_test}) 
                test_batch_scores.append(test_accuracy) #change this line to get avg
                
                #Save dataframe of predictions for test data
                predictions_sigm = logits.eval(feed_dict = {features_input:X_test}) #get predictions from the test data features
                temp_df = pd.DataFrame(predictions_sigm, columns = col_names[:-1]) #put these in a temp dataframe
                true_class = np.argmax(y_test, axis = 1)     #This saves the true class from test data labels
                temp_df['True class'] = true_class        #This adds true class to the temp_df
                test_predictions = test_predictions.append(temp_df, ignore_index = True)          #append the temp df to the full df     ##############
                #print(test_pred.shape)
                #save as a csv for each epoch
              
              # Get test accuracy for this epoch
              test_avg_score = Average(test_batch_scores) # gets the average across all val minibatches for this epoch
              test_accuracy_scores.append(test_avg_score) # appends this average to a list containing the avg for each epoch
              print('New validation accuracy benchmark, test accuracy:', test_avg_score)#accuracy.eval({features_input:X_test, labels_input:y_test})) #TF is smart so just knows to feed it through the model without us seeming to tell it to. .eval() uses the current session which I guess is my model?
              #print(test_predictions)
              # Save test predictions for this epoch
              # names file with validation accuracy, not test accuracy. Final test accuracy should be determine by taking the mode prediction per min.
            else:
              # If validation accuracy did not improve
              print('No validation accuracy improvement, test accuracy still: ' + str(test_avg_score))  
    
    # Save test data predictions for the epoch which had the highest val data accuracy
    np.savetxt(results_dir + "CrossValRepeat" + str(repeat) + "_Epoch" + best_epoch + "_ValAcc_" + str(round(highest_acc_score, 5)) + ".csv",
                test_predictions, delimiter = ",") #put 'r"r'C:\Users\...\test_predictions' to save in a different folder
    print('Highest validation accuracy: ' + str(highest_acc_score))

tf.app.run(main)   

W1018 11:30:33.696923 140320038377344 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
W1018 11:30:33.783624 140320038377344 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:1666: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
W1018 11:30:33.858688 140320038377344 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/tensorflow_core/python/ops/nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
I1018 11:30:37.189760 140320038377344 saver.py:

mini batch0
Epoch 1 completed out of 2 , loss: 0.7162290811538696
Validation accuracy: 0.5702764987945557
New validation accuracy benchmark, test accuracy: 0.4000000059604645
mini batch0
Epoch 2 completed out of 2 , loss: 0.7042340636253357
Validation accuracy: 0.5714285969734192
New validation accuracy benchmark, test accuracy: 0.4000000059604645
Highest validation accuracy: 0.5714285969734192


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
